## Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

In [1]:
# Импортируем необходимые библиотеки и Spark-сессию
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.sql.functions import isnan, when, count, col

In [2]:
spark = SparkSession.builder \
    .appName("Linear Regression Project") \
    .getOrCreate()

# Подготовка данных

In [3]:
# Прочтем содержимое файла housing.csv
file_path = "/datasets/housing.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [4]:
# Вывод типов данных столбцов набора данных
df.printSchema()
print()
df.show(5)

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)


+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR 

In [5]:
# Подсчет количества пропущенных значений или значений NaN в каждом столбце.
missing_values = df.select([count(when(isnan(col_name) | col(col_name).isNull(), 
                                       col_name)).alias(col_name) for col_name in df.columns])

# Отображение результатов
missing_values.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|        0|       0|                 0|          0|           207|         0|         0|            0|                 0|              0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



In [6]:
# Обработка пропущенных значений
df = df.na.fill({'total_bedrooms': 0})  # Заменим отсутствующие значения на 0

print()

missing_values = df.select([count(when(isnan(col_name) | col(col_name).isNull(), 
                                       col_name)).alias(col_name) for col_name in df.columns])
missing_values.show()


+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|        0|       0|                 0|          0|             0|         0|         0|            0|                 0|              0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



In [7]:
# Разделим данные на наборы для обучения и тестирования.
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

# Подгонка StringIndexer к обучающим данным
string_indexer = StringIndexer(inputCol="ocean_proximity", outputCol="ocean_index", handleInvalid="keep").fit(train_df)

# Трансформация данных обучения и тестирования
indexed_train_df = string_indexer.transform(train_df)
indexed_test_df = string_indexer.transform(test_df)

# Подгонка OneHotEncoder к обучающим данным
encoder = OneHotEncoder(inputCols=["ocean_index"], outputCols=["ocean_encoded"]).fit(indexed_train_df)

# Трансформация данных обучения и тестирования
encoded_train_df = encoder.transform(indexed_train_df)
encoded_test_df = encoder.transform(indexed_test_df)

# Обучение моделей

*Построение двух моделей линейной регрессии для разных наборов данных*

In [8]:
# Создадим вектор признаков
feature_cols = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms",
                "population", "households", "median_income", "ocean_encoded"]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
feature_vector_df = assembler.transform(encoded_train_df)

# Создадим модель линейной регрессии
lr = LinearRegression(labelCol="median_house_value", regParam=0.01)

# Подгонка модели к обучающим данным
lr_model_all_data = lr.fit(feature_vector_df)

# Сделаем прогнозы на данных обучения
predictions_all_data = lr_model_all_data.transform(feature_vector_df)

# Создадим вектор признаков для числовых данных
numerical_cols = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms",
                  "population", "households", "median_income"]

assembler_numerical = VectorAssembler(inputCols=numerical_cols, outputCol="features")
numerical_vector_df = assembler_numerical.transform(df)

# Создадим модель линейной регрессии для числовых данных
lr_numerical = LinearRegression(labelCol="median_house_value", regParam=0.01)

# Подгонка модели к числовым данным
lr_model_numerical = lr_numerical.fit(numerical_vector_df)

# Сделаем прогнозы на числовых данных
predictions_numerical = lr_model_numerical.transform(numerical_vector_df)

23/07/07 03:35:53 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/07/07 03:35:53 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/07/07 03:35:54 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/07/07 03:35:54 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


# Анализ результатов

In [9]:
# Сравним результаты линейной регрессии на двух наборах данных для показателей RMSE, MAE и R2
evaluator = RegressionEvaluator(labelCol="median_house_value")

rmse_all_data = evaluator.evaluate(predictions_all_data, {evaluator.metricName: "rmse"})
rmse_numerical = evaluator.evaluate(predictions_numerical, {evaluator.metricName: "rmse"})

mae_all_data = evaluator.evaluate(predictions_all_data, {evaluator.metricName: "mae"})
mae_numerical = evaluator.evaluate(predictions_numerical, {evaluator.metricName: "mae"})

r2_all_data = evaluator.evaluate(predictions_all_data, {evaluator.metricName: "r2"})
r2_numerical = evaluator.evaluate(predictions_numerical, {evaluator.metricName: "r2"})

In [10]:
print("Результаты линейной регрессии:")
print("Используя все данные — RMSE: {:.2f}".format(rmse_all_data))
print("Использование числовых переменных — RMSE: {:.2f}".format(rmse_numerical))
print("")

print("Используя все данные - MAE: {:.2f}".format(mae_all_data))
print("Использование числовых переменных - MAE: {:.2f}".format(mae_numerical))
print("")

print("Используя все данные - R2: {:.2f}".format(r2_all_data))
print("Использование числовых переменных - R2: {:.2f}".format(r2_numerical))

Результаты линейной регрессии:
Используя все данные — RMSE: 68332.87
Использование числовых переменных — RMSE: 69748.19

Используя все данные - MAE: 49623.77
Использование числовых переменных - MAE: 51051.26

Используя все данные - R2: 0.65
Использование числовых переменных - R2: 0.63


**Выводы:**  

- Эти метрики показывают, насколько хорошо модели линейной регрессии соответствуют данным. Более низкие значения RMSE и MAE указывают на лучшую производительность модели, а более высокое значение R2 указывает на лучшее соответствие данным.

- Основываясь на результатах, кажется, что использование всех доступных признаков (включая категориальные признаки) приводит к несколько лучшей производительности по сравнению с использованием только числовых переменных. Однако различия в производительности относительно невелики.

In [11]:
# Остановим сессию Spark
spark.stop()

<font color='green'>Немного полезного материала:
+  https://sparkbyexamples.com/pyspark-tutorial/, https://sparkbyexamples.com/
+  https://github.com/dvgodoy/handyspark
+  https://www.tutorialspoint.com/pyspark/index.htm
+  https://www.guru99.com/pyspark-tutorial.html
+  https://databricks.com/spark/getting-started-with-apache-spark/machine-learning#load-sample-data
</font>